| Version | Published Date| Details |
| -- | -- | -- |
| ver.1.0.0 | 2023/8/29 | 初版 |

# 正規表現についてもっと詳しくなろう

Station1 では強力なツールである **正規表現** について学びました。まずは数字のパターンを検出できるようになりましたが，このStationではより詳しく正規表現を学びます。正規表現は書き方もややむずかしく，学ぶこともより増えますが，引き続きしっかり習得していきましょう。

# 正規表現によるパターンマッチの続き

## `()` を用いたグルーピング

前回のStationでも `group()` を使いました。これは正規表現の機能のひとつです。

電話番号を市外局番とそれ以降に分けて取得するとしましょう。その場合は `(\d\d\d\d)-(\d\d-\d\d\d\d)` のように正規表現に `()` を追加してグループを作成します。そして `group()` ごとにマッチした文字列を取得します。

`()` で囲まれたグループごとにグループ1, 2, ... となります。`group()` メソッドに `1` や `2` を渡すとマッチした文字列の異なる部分を取得できます。`0` を渡すかなにも渡さなければマッチした文字列全体が返ります。

In [ ]:
import re

今回もまずは `re` を `import` します。

In [ ]:
phone_num_regex = re.compile(r'(\d\d\d\d)-(\d\d-\d\d\d\d)')
mo = phone_num_regex.search('私の電話番号は0120-56-7890です。')

In [ ]:
mo.group(1)

'0120'

In [ ]:
mo.group(2)

'56-7890'

In [ ]:
mo.group()

'0120-56-7890'

`groups()` を使うとすべてのグループを一気に取得できます。`group()` と違って `s` がつく `groups()` になっていることに注意しましょう。

In [ ]:
mo.groups()

('0120', '56-7890')

`groups()` は複数の値からなるタプルを返すため，このように複数代入を行うこともできます。

In [ ]:
area_code, main_number = mo.groups()

In [ ]:
area_code

'0120'

In [ ]:
main_number

'56-7890'

正規表現では `()` は特別な意味を持ちます。では `()` そのものを検索したい場合はどうすればよいでしょうか？たとえば市外局番が `()` を使って書かれていることもあります。このような場合にはバックスラッシュ `\` を使って `()` を **エスケープ (Escape)** する必要があります。

In [ ]:
phone_num_regex = re.compile(r'(\(\d\d\d\d\)) (\d\d-\d\d\d\d)')
mo = phone_num_regex.search('私の電話番号は (0120) 23-4234 です。')
mo.group(1)

'(0120)'

In [ ]:
mo.group(2)

'23-4234'

正規表現では以下の文字は特別な意味を持ちます。これらの文字を用いる場合は同様に `\` を用いるようにしましょう。

```
.  ^  $  *  +  ?  {  }  [  ]  \\  |  (  )
```

## `|` を用いた複数グループとのマッチ

`|` を用いると，複数パターンのうちのひとつとマッチできます。たとえば `r'Tokyo|Osaka'` という正規表現は `Tokyo` または `Osaka` とマッチします。すべてのマッチした箇所を取得する `finaall()` メソッドについては後で解説します。

検索対象に `Tokyo` と `Osaka` の両方がある場合，先に出現した方が `Match` オブジェクトとして返ります。

In [ ]:
city_regex = re.compile(r'Tokyo|Osaka')
mo1 = city_regex.search('Tokyo and Osaka')
mo1.group()

'Tokyo'

In [ ]:
mo2 = city_regex.search('Osaka and Tokyo')
mo2.group()

'Osaka'

縦線は，複数パターンのうち1つにマッチするためにも使えます。たとえば `Tokyo` `Tokai` `Tokio` `Tokiwa` のいずれかにマッチさせたい場合を考えます。この場合すべて `Tok` からはじまるため，接頭辞として一度だけマッチすればよいでしょう。

In [ ]:
tok_regex = re.compile(r'Tok(yo|ai|io|iwa)')
mo = tok_regex.search("Tokyo station is very beautiful.")
mo.group()

'Tokyo'

In [ ]:
mo.group(1)

'yo'

引数をつけずに `group()` を呼び出すとマッチした文字列全体を，`group(1)` のように呼び出すと1番目 `()` のグループ内にマッチした文字列だけを取得できます。

## `?` を用いた任意のマッチ

マッチしてもしなくてもよいパターンを指定したい場合があります。つまりテキストの一部があってもなくてもいい場合です。 `?` を用いるとその直前のパターンを任意パターンとして指定します。

In [ ]:
tokyo_regex = re.compile(r'Toky(ot)?o')
mo1 = tokyo_regex.search('Tokyo')
mo1.group()

'Tokyo'

In [ ]:
mo2 = tokyo_regex.search('Tokyoto')
mo2.group()

'Tokyoto'

正規表現の `(ot)?` の部分は `ot` というパターンが任意であることを意味します。そのため正規表現は `ot` が0回または1回現れるテキストにマッチします。そのため `Tokyo` にも `Tokyoto` にもマッチします。

市外局番のあるなしに関わらず電話番号を検索したいとしましょう。その場合はこのように書けます。

In [ ]:
phone_num_regex = re.compile(r'(\d\d\d\d-)?\d\d-\d\d\d\d')
mo1 = phone_num_regex.search('私の電話番号は 0120-23-4234 です。')
mo1.group()

'0120-23-4234'

In [ ]:
mo2 = phone_num_regex.search('私の電話番号は 23-4234 です。')
mo2.group()

'23-4234'

同様にこの場合は `?` は「直前のグループに0または1回マッチする」とみなせます。

## `*` を用いた0回以上のマッチ

`*` は「0回以上にマッチする」という意味です。アスタリスクの直前のグループが任意の回数出現してもよいということです。まったく出現しなくてもよいし，繰り返し出現しても問題ありません。

In [ ]:
tokyo_regex = re.compile(r'Toky(ot)*o')
mo1 = tokyo_regex.search('Tokyo is the capital of Japan.')
mo1.group()

'Tokyo'

In [ ]:
mo2 = tokyo_regex.search('Tokyoto is the capital of Japan.')
mo2.group()

'Tokyoto'

In [ ]:
mo3 = tokyo_regex.search('Tokyototototo is the capital of Japan.')
mo3.group()

'Tokyototototo'

`Tokyo` に関しては `(ot)*` の正規表現が0回，`Tokyoto` に関しては1回，`Tokyototototo` に関しては4回マッチします。

## `+` を用いた1回以上のマッチ

`*` が0回以上にマッチするのに対して `+` は「1回以上マッチ」することを意味します。プラスは少なくとも1回出現する必要があります。

In [ ]:
tokyo_regex = re.compile(r'Toky(ot)+o')
mo1 = tokyo_regex.search('Tokyoto is the capital of Japan.')
mo1.group()

'Tokyoto'

In [ ]:
mo2 = tokyo_regex.search('Tokyototototo is the capital of Japan.')
mo2.group()

'Tokyototototo'

In [ ]:
mo3 = tokyo_regex.search('Tokyo is the capital of Japan.')
mo3.group()

AttributeError: ignored

`mo3` にマッチするものがないため `group()` を呼ぶとエラーになります。エラーコードを読みましょう。 *'NoneType' object has no attribute 'group'* とは「 `None` の型のオブジェクトには `group` という属性はありません」という意味です。

## `{}` による繰り返し回数の指定

`{}` を使うとグループの繰り返し出現回数を指定できます。たとえば `(Wa){3}` という正規表現は `WaWaWa` という文字列にマッチしますが `WaWa` にはマッチしません。`Wa` が2回しか現れないためです。

繰り返し回数は単一の数字だけでなく，カンマ区切りで最大値と最小値の指定が可能です。たとえば `(Ma){3,5}` は `MaMaMa` にも `MaMaMaMa` にも `MaMaMaMaMa` にもマッチします。

片方を省略すると最小値と最大値を指定しないことになります。たとえば `(Wa){3,}` とすると `(Wa)` が3回以上， `(Wa){5,}` とすると0~5回出現する場合にマッチします。

`{}` を使うと正規表現を短く書くのに役立ちます。`(Ma){3}` と `MaMaMa` ， `(Ma){3,5}` と `MaMaMa|MaMaMaMa|MaMaMaMaMa` はそれぞれ同じ意味です。

In [ ]:
ha_regex = re.compile(r'(Ha){3}')
mo1 = ha_regex.search('HaHaHa')
mo1.group()

'HaHaHa'

In [ ]:
mo2 = ha_regex.search('Ha')
mo2.group()

AttributeError: ignored

これも `Ha` ひとつだけではマッチしないため，同じようにエラーになります。

## 貪欲マッチと非貪欲マッチ

ところで `MaMaMaMaMa` という文字列に対して `(Ma){3,5}` というパターンは `Ma` が3回，4回，5回のいずれの場合にもマッチします。先の例で `group()` を呼び出すと，少ない回数でマッチせずに最大回数にマッチした `MaMaMaMaMa` が返ってきます。 `MaMaMa` も `MaMaMaMa` も `(Ma){3,5}` に正しくマッチするにも関わらず，です。

これはなぜでしょうか？なぜならPythonの正規表現はデフォルトでは **貪欲 (Greedy)** にマッチするためです。つまり複数の可能性のあるものは最も長いものにマッチします。一方で `)` の後に `?` をつけると **怠惰 (Lazy)** あるいは **非貪欲** なマッチを意味し，最も短いものにマッチするようになります。

「貪欲」と「怠惰」というのはは、コンピューターサイエンスにおいてアルゴリズムの設計でよく使われる用語です。興味がある方はそれぞれの言葉を調べてみてください。

In [ ]:
greedy_ha_regex = re.compile(r'(Ha){3,5}')
mo1 = greedy_ha_regex.search(r'HaHaHaHaHa')
mo1.group()

'HaHaHaHaHa'

In [ ]:
lazy_ha_regex = re.compile(r'(Ha){3,5}?')
mo2 = lazy_ha_regex.search(r'HaHaHaHaHa')
mo2.group()

'HaHaHa'

正規表現において `?` が「任意グループの指定」と「非貪欲マッチ」の2種類の意味を持つことに注意しましょう。

## `findall()` メソッド

今までは `Regex` オブジェクトにおける `search()` を主に用いてきましたが，他に `findall()` メソッドがあります。

In [ ]:
phone_num_regex = re.compile(r'\d\d-\d\d\d\d')
mo = phone_num_regex.search('自宅の電話番号は 55-4433 です。会社の電話番号は 23-4234 です。')
mo.group()

'55-4433'

`search()` は最初に見つかった `Match` オブジェクトを返すのに対して `findall()` は文字列のリストを返します。リストの各要素はマッチした文字列です。

In [ ]:
phone_num_regex = re.compile(r'\d\d-\d\d\d\d')
phone_num_regex.findall('自宅の電話番号は 55-4433 です。会社の電話番号は 23-4234 です。')

['55-4433', '23-4234']

正規表現にグループが含まれている場合 `findall()` はタプルのリストを返します。各タプルの要素は，正規表現のグループに対応しマッチした文字列です。 `findall()` の動作を確認しましょう。

In [ ]:
phone_num_regex = re.compile(r'(\(\d\d\d\d\)) (\d\d-\d\d\d\d)')
mo = phone_num_regex.findall('自宅の電話番号は (0345) 55-4433 です。会社の電話番号は (0120) 23-4234 です。')
mo

[('(0345)', '55-4433'), ('(0120)', '23-4234')]

`findall()` メソッドの返り値についてのまとめです。

- グループのない正規表現の場合，マッチした文字列のリストを返す。
- グループのある正規表現の場合，グループに対応した文字列のタプルのリストを返す。

# 確認テスト

- 東京都の電話番号にマッチする正規表現を書いてください。東京都の電話番号とは `03-1234-5678` のように市外局番 `03` のあとに数字4ケタ，ハイフン，数字4ケタの順で続くものとします。
- `東京` と `京都` と `京` と `東京都` にマッチする正規表現を書いてください。

In [ ]:
# 確認テスト (1)
import re
mo = re.search(
    r'03-\d\d\d\d-\d\d\d\d'# <- ここに正規表現を書きます (ここに記入した内容を回答フォームに登録してください)
    , '東京オフィス:03-1234-5678、大阪オフィス:06-1234-5678')
print(f'マッチした電話番号が見つかりました: {mo.group()}')

マッチした電話番号が見つかりました: 03-1234-5678


In [ ]:
# 確認テスト (2)
import re
mo = re.findall(
    r'(東?京{1}都?)'# <- ここに正規表現を書きます (ここに記入した内容を回答フォームに登録してください)
    , '弊社は東京オフィスが東京都千代田区にあり、京都オフィスがプラザ京にあります。')
print(f'マッチした文字列が見つかりました: {mo}')

マッチした文字列が見つかりました: ['東京', '東京都', '京都', '京']


# 新しいセクション